# Preparing Data

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

In [ ]:
vaccine = pd.read_csv('vaccine.csv')
vaccine

In [ ]:
# Converting pandas dataframe to Dataset
vaccine = vaccine.rename(columns={'decision': 'label', 'response': 'text'})
vaccine = Dataset.from_pandas(vaccine)
vaccine

In [ ]:
model_ckpt = 'distilbert-base-uncased'

# Tokenizing the dataset
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
tokenize = lambda batch: tokenizer(batch['text'], padding="max_length", truncation=True)
vaccine = vaccine.map(tokenize, batched=True)

# Feature Extraction

In [ ]:
import torch
from transformers import AutoModel

In [ ]:
# Loading the model and moving it to the GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = AutoModel.from_pretrained(model_ckpt).to(device)
model

In [ ]:
def extract_features(batch):
    """Extract features from a batch of items"""
    inputs = {k:v.to(device) for k, v in batch.items() if k in tokenizer.model_input_names}
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
        return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}


vaccine = vaccine.map(extract_features, batched=True, batch_size=5)
embeds = pd.DataFrame(vaccine['hidden_state'])
embeds

# Predicting vaccine decisions with embeddings

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split

In [ ]:
clf = LogisticRegressionCV()
X_train, X_test, y_train, y_test = train_test_split(embeds, vaccine['label'])
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

# Pedicting vaccine decisions the LM fine-tuning

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
vaccine = vaccine.train_test_split(test_size=0.2)
vaccine

In [ ]:
# Training the model
compute_metrics = lambda x: x
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=vaccine['train'],
    eval_dataset=vaccine['test'],
    compute_metrics=compute_metrics,
)
trainer.train()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_labels = len(vaccine['decision'].unique())
model = (
    AutoModelForSequenceClassification
    .from_pretrained(model_ckpt, num_labels=num_labels)
    .to_device(device)
)